<a href="https://colab.research.google.com/github/Prikshit7766/nlp-test-implementation/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
cd /content/drive/MyDrive/nlp test implementation

/content/drive/MyDrive/nlp test implementation


In [15]:
! pip install -q pyspark==3.3.0 spark-nlp==4.3.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.2/473.2 kB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 20.1 MB/s eta 0:00:00


In [16]:
!pip install nlptest

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.4/809.4 kB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the dataframe from a CSV file
df = pd.read_csv('FinalBalancedDataset.csv')


In [3]:
df.shape

(56745, 3)

In [4]:
df.isna().sum()

Unnamed: 0    0
Toxicity      0
tweet         0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56745 entries, 0 to 56744
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  56745 non-null  int64 
 1   Toxicity    56745 non-null  int64 
 2   tweet       56745 non-null  object
dtypes: int64(2), object(1)
memory usage: 1.3+ MB


In [6]:
df.head()

,Unnamed: 0,Toxicity,tweet
0,0,0,@user when a father is dysfunctional and is s...
1,1,0,@user @user thanks for #lyft credit i can't us...
2,2,0,bihday your majesty
3,3,0,#model i love u take with u all the time in ...
4,4,0,factsguide: society now #motivation


In [7]:
df = df.rename(columns={'Toxicity': 'label'})

In [8]:
# Count the occurrences of each class in the target column
class_counts = df['label'].value_counts()

In [9]:
# Determine the minimum count among the classes
min_count = class_counts.min()

# Limit the minimum count to 2,500 or the available count, whichever is smaller
min_count = min(min_count, 2500)

In [10]:
# Trim the data to have an equal number of records for each class
trimmed_data = pd.concat([df[df['label'] == cls].sample(min_count) for cls in class_counts.index])

# Randomly sample 5,000 records from the trimmed data
trimmed_data = trimmed_data.sample(n=5000, random_state=42)


In [11]:
trimmed_data=trimmed_data.drop("Unnamed: 0",axis=1)

In [12]:
# Save the trimmed data as a CSV file
trimmed_data.to_csv('trimmed_data.csv', index=False)

spark nlp

In [17]:
import os
import sys

import sparknlp

from sparknlp.base import *
from sparknlp.common import *
from sparknlp.annotator import *

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

import pandas as pd
spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

spark

Spark NLP version:  4.3.2
Apache Spark version:  3.3.0


In [18]:
import pandas as pd
data = pd.read_csv('trimmed_data.csv') 

In [19]:
spark_df = spark.createDataFrame(data)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [20]:
spark_df.show()

+-----+--------------------+
|label|               tweet|
+-----+--------------------+
|    0|wow september 1 2...|
|    1|RT @UrbanCrazines...|
|    1|RT @RuNeshaShamia...|
|    0|i am thankful for...|
|    0|justice has been ...|
|    0|#diasoleado ðð...|
|    0|#twinkletwinkle t...|
|    0|5 more days until...|
|    0|@user oh dear, is...|
|    0|#visiting   goril...|
|    0|this makes me #ma...|
|    0|just been to view...|
|    0| @user just read ...|
|    0|#lucky   #dad to ...|
|    1|council staff hav...|
|    1|@SlothNG i can gu...|
|    0|i am fantastic. #...|
|    1|@chandraingram95 ...|
|    0|These peanut butt...|
|    0|@user the dance m...|
+-----+--------------------+
only showing top 20 rows



In [21]:
spark_df.take(2)

[Row(label=0, tweet='wow september 1 2015 me '),
 Row(label=1, tweet='RT @UrbanCraziness: "nah go talk to ur hoes since u dont wanna text back n shit" http://t.co/cvBzmjQXyO')]

In [22]:
from pyspark.sql.functions import col

spark_df.groupBy("label") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

    #Label indicating whether a tweet is toxic(1) or not(0).

+-----+-----+
|label|count|
+-----+-----+
|    0| 2500|
|    1| 2500|
+-----+-----+



## Building  Pipeline

### LogReg with CountVectorizer

In [23]:
from pyspark.ml.feature import CountVectorizer, HashingTF, IDF, OneHotEncoder, StringIndexer, VectorAssembler, SQLTransformer

In [ ]:
%%time

document_assembler = DocumentAssembler() \
      .setInputCol("tweet") \
      .setOutputCol("document")
    
tokenizer = Tokenizer() \
      .setInputCols(["document"]) \
      .setOutputCol("token")
      
normalizer = Normalizer() \
      .setInputCols(["token"]) \
      .setOutputCol("normalized") # remove special characters

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

stemmer = Stemmer() \
      .setInputCols(["cleanTokens"]) \
      .setOutputCol("stem")

finisher = Finisher() \
      .setInputCols(["stem"]) \
      .setOutputCols(["token_features"]) \
      .setOutputAsArray(True) \
      .setCleanAnnotations(False) #The Finisher annotator helps you to clean the metadata (if it’s set to true) and output the results into an array

countVectors = CountVectorizer(inputCol="token_features", outputCol="features", vocabSize=10000, minDF=5)


nlp_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            stemmer, 
            finisher,
            countVectors,
            ])

nlp_model = nlp_pipeline.fit(spark_df)

processed = nlp_model.transform(spark_df)

processed.count()

CPU times: user 190 ms, sys: 42.3 ms, total: 232 ms
Wall time: 12.4 s


5000

In [ ]:
processed.select('tweet','token_features').show(truncate=50)

+--------------------------------------------------+--------------------------------------------------+
|                                             tweet|                                    token_features|
+--------------------------------------------------+--------------------------------------------------+
|the sellers accepted my offer. and the process ...|           [seller, accept, offer, process, begin]|
|the microwave at work is broken, so my ghetto a...|[microwav, work, broken, ghetto, ass, put, hamb...|
|RT @Oz_lito: @1stBlocJeremiah ctfu u bloody but...|[rt, ozlito, stblocjeremiah, ctfu, u, bloodi, y...|
|it's so amazing how it's hard for disadvantaged...|[amaz, hard, disadvantag, footbal, settl, team,...|
|          happy day!  #altwaystoheal  #healthy    |                 [happi, dai, altwaystoh, healthi]|
|people who can get time off work to go to the e...|[peopl, get, time, work, go, euro, havent, gone...|
|@user the library of alexandria wasn't the only...|[user, libra

In [ ]:
processed.select('token_features').take(2)

[Row(token_features=['seller', 'accept', 'offer', 'process', 'begin']),
 Row(token_features=['microwav', 'work', 'broken', 'ghetto', 'ass', 'put', 'hamburg', 'patti', 'toaster', 'warm'])]

In [ ]:
processed.select('features').take(2)

[Row(features=SparseVector(1315, {444: 1.0, 710: 1.0, 1077: 1.0})),
 Row(features=SparseVector(1315, {17: 1.0, 50: 1.0, 121: 1.0, 258: 1.0, 1099: 1.0}))]

In [ ]:
processed.select('tweet','features','label').show()

+--------------------+--------------------+-----+
|               tweet|            features|label|
+--------------------+--------------------+-----+
|the sellers accep...|(1315,[444,710,10...|    0|
|the microwave at ...|(1315,[17,50,121,...|    1|
|RT @Oz_lito: @1st...|(1315,[1,2,7,147,...|    1|
|it's so amazing h...|(1315,[60,192,194...|    0|
|  happy day!  #al...|(1315,[11,16,120,...|    0|
|people who can ge...|(1315,[8,18,25,27...|    0|
|@user the library...|(1315,[0,23,432,7...|    0|
|somewhere in my t...|(1315,[77,108,153...|    0|
|back home k.lbvr ...|(1315,[6,49,94,10...|    0|
|hating the conser...|(1315,[34,61,62,8...|    0|
|Going up to the v...|(1315,[18,39,352,...|    0|
|@user lolol i'm n...|(1315,[0,5,87,139...|    0|
|@CallNeeshCakey p...|(1315,[238,327,35...|    0|
|#petal   polar be...|(1315,[66,196,234...|    0|
|Love getting nude...|(1315,[1,6,8,429]...|    1|
|You a hoe if you ...|(1315,[3,4,784,79...|    1|
|best #lawofattrac...|(1315,[75,120,268...|    0|


In [ ]:
# set seed for reproducibility
(trainingData, testData) = processed.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 3478
Test Dataset Count: 1522


In [ ]:
trainingData.printSchema()

root
 |-- label: long (nullable = true)
 |-- tweet: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- token: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value:

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0)

lrModel = lr.fit(trainingData)

predictions = lrModel.transform(testData)

predictions.filter(predictions['prediction'] == 0) \
    .select("tweet",'label',"probability","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)



+------------------------------+-----+------------------------------+----------+
|                         tweet|label|                   probability|prediction|
+------------------------------+-----+------------------------------+----------+
|this is very good camp food...|    0|[0.9669286261089691,0.03307...|       0.0|
|have a great weekend, frien...|    0|[0.9663494727116793,0.03365...|       0.0|
|view for lunch ð #spain ...|    0|[0.9639490023980476,0.03605...|       0.0|
| @user make sure you regist...|    0|[0.9607156733538808,0.03928...|       0.0|
|travel to me   #inshot #gir...|    0|[0.9547928449600995,0.04520...|       0.0|
|serious holiday blues!first...|    0|[0.9538267459821224,0.04617...|       0.0|
|whats happening here... pls...|    0|[0.9536519186543062,0.04634...|       0.0|
|a small yet super treat aft...|    0|[0.952650344938334,0.047349...|       0.0|
|#ana   bull hill climb: you...|    0|[0.9519699899803007,0.04803...|       0.0|
| @user omg i won't be able 

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")

evaluator.evaluate(predictions)

0.8883048620236531

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
y_true = predictions.select("label")
y_true = y_true.toPandas()

y_pred = predictions.select("prediction")
y_pred = y_pred.toPandas()

In [ ]:
y_pred.prediction.value_counts()

0.0    772
1.0    750
Name: prediction, dtype: int64

In [ ]:
cnf_matrix = confusion_matrix(list(y_true.label.astype(int)), list(y_pred.prediction.astype(int)))
cnf_matrix

array([[676,  74],
       [ 96, 676]])

In [ ]:
print(classification_report(y_true.label, y_pred.prediction))
print(accuracy_score(y_true.label, y_pred.prediction))

              precision    recall  f1-score   support

           0       0.88      0.90      0.89       750
           1       0.90      0.88      0.89       772

    accuracy                           0.89      1522
   macro avg       0.89      0.89      0.89      1522
weighted avg       0.89      0.89      0.89      1522

0.8883048620236531


## LogReg with TFIDF

In [ ]:
from pyspark.ml.feature import HashingTF, IDF

hashingTF = HashingTF(inputCol="token_features", outputCol="rawFeatures", numFeatures=10000)

idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms

nlp_pipeline_tf = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            stemmer, 
            finisher,
            hashingTF,
            idf,
            ])

nlp_model_tf = nlp_pipeline_tf.fit(spark_df)

processed_tf = nlp_model_tf.transform(spark_df)

processed_tf.count()

5000

In [ ]:
# set seed for reproducibility
processed_tf.select('tweet','features','label').show()

+--------------------+--------------------+-----+
|               tweet|            features|label|
+--------------------+--------------------+-----+
|the sellers accep...|(10000,[1513,2657...|    0|
|the microwave at ...|(10000,[770,3776,...|    1|
|RT @Oz_lito: @1st...|(10000,[701,865,2...|    1|
|it's so amazing h...|(10000,[468,874,1...|    0|
|  happy day!  #al...|(10000,[353,5285,...|    0|
|people who can ge...|(10000,[696,1149,...|    0|
|@user the library...|(10000,[1233,1991...|    0|
|somewhere in my t...|(10000,[55,349,52...|    0|
|back home k.lbvr ...|(10000,[969,1568,...|    0|
|hating the conser...|(10000,[15,2069,2...|    0|
|Going up to the v...|(10000,[2028,4416...|    0|
|@user lolol i'm n...|(10000,[1678,2182...|    0|
|@CallNeeshCakey p...|(10000,[2952,6409...|    0|
|#petal   polar be...|(10000,[253,454,4...|    0|
|Love getting nude...|(10000,[701,5900,...|    1|
|You a hoe if you ...|(10000,[2712,3330...|    1|
|best #lawofattrac...|(10000,[353,763,2...|    0|


In [ ]:
(trainingData, testData) = processed_tf.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 3478
Test Dataset Count: 1522


In [ ]:
lrModel_tf = lr.fit(trainingData)

predictions_tf = lrModel_tf.transform(testData)

predictions_tf.select("tweet","label","probability","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+------------------------------+-----+------------------------------+----------+
|                         tweet|label|                   probability|prediction|
+------------------------------+-----+------------------------------+----------+
|this is very good camp food...|    0|[0.9687570456086259,0.03124...|       0.0|
|a small yet super treat aft...|    0|[0.957801237378701,0.042198...|       0.0|
|pause ð #goodmorning #ha...|    0|[0.9557869861490598,0.04421...|       0.0|
|check  steve kerr rips offi...|    0|[0.9509212051985791,0.04907...|       0.0|
|whats happening here... pls...|    0|[0.9475181156488829,0.05248...|       0.0|
| @user make sure you regist...|    0|[0.9474741661899445,0.05252...|       0.0|
|amazing 1st week with a new...|    0|[0.9472651405161227,0.05273...|       0.0|
|take yourself away sometime...|    0|[0.9472601685410799,0.05273...|       0.0|
|serious holiday blues!first...|    0|[0.946126846216106,0.053873...|       0.0|
|In-N-Out is like the sunshi

In [ ]:
y_true = predictions_tf.select("label")
y_true = y_true.toPandas()

y_pred = predictions_tf.select("prediction")
y_pred = y_pred.toPandas()

print(classification_report(y_true.label, y_pred.prediction))
print(accuracy_score(y_true.label, y_pred.prediction))

              precision    recall  f1-score   support

           0       0.86      0.87      0.87       750
           1       0.88      0.86      0.87       772

    accuracy                           0.87      1522
   macro avg       0.87      0.87      0.87      1522
weighted avg       0.87      0.87      0.87      1522

0.8679369250985546


## Random Forest with TFIDF

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol="label", \
                            featuresCol="features", \
                            numTrees = 100, \
                            maxDepth = 4, \
                            maxBins = 32)

# Train model with Training Data
rfModel = rf.fit(trainingData)
predictions_rf = rfModel.transform(testData)

In [ ]:
predictions_rf.select("tweet","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+------------------------------+------------------------------+-----+----------+
|                         tweet|                   probability|label|prediction|
+------------------------------+------------------------------+-----+----------+
|thank you lord for this bea...|[0.5828660171374689,0.41713...|    0|       0.0|
| @user @user   thursday #lo...|[0.577210260625392,0.422789...|    0|       0.0|
|  mornin folks have a #posi...|[0.5769128262281499,0.42308...|    0|       0.0|
| @user tomorrow i meet up w...|[0.5762152691262067,0.42378...|    0|       0.0|
|love around me, i love my g...|[0.5761596875209244,0.42384...|    0|       0.0|
| @user when they nap on you...|[0.5741740834178458,0.42582...|    0|       0.0|
|  lovely gift on a sunny da...|[0.5739713235545756,0.42602...|    0|       0.0|
| â #wti drops to lows nea...|[0.5727125788658238,0.42728...|    0|       0.0|
| â #new zealand current a...|[0.5703414959230629,0.42965...|    0|       0.0|
|@user &lt;3 &lt;3 listen to

In [ ]:
y_true = predictions_rf.select("label")
y_true = y_true.toPandas()

y_pred = predictions_rf.select("prediction")
y_pred = y_pred.toPandas()

print(classification_report(y_true.label, y_pred.prediction))
print(accuracy_score(y_true.label, y_pred.prediction))

              precision    recall  f1-score   support

           0       0.80      0.93      0.86       750
           1       0.92      0.77      0.84       772

    accuracy                           0.85      1522
   macro avg       0.86      0.85      0.85      1522
weighted avg       0.86      0.85      0.85      1522

0.8488830486202366


## LogReg with Spark NLP Glove Word Embeddings

In [ ]:
document_assembler = DocumentAssembler() \
      .setInputCol("tweet") \
      .setOutputCol("document")
    
tokenizer = Tokenizer() \
      .setInputCols(["document"]) \
      .setOutputCol("token")
    
normalizer = Normalizer() \
      .setInputCols(["token"]) \
      .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

glove_embeddings = WordEmbeddingsModel().pretrained() \
      .setInputCols(["document",'cleanTokens'])\
      .setOutputCol("embeddings")\
      .setCaseSensitive(False)

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "embeddings"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")
    
embeddings_finisher = EmbeddingsFinisher() \
      .setInputCols(["sentence_embeddings"]) \
      .setOutputCols(["finished_sentence_embeddings"]) \
      .setOutputAsVector(True)\
      .setCleanAnnotations(False)

explodeVectors = SQLTransformer(statement=
      "SELECT EXPLODE(finished_sentence_embeddings) AS features, * FROM __THIS__")



nlp_pipeline_w2v = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            glove_embeddings,
            embeddingsSentence,
            embeddings_finisher,
            explodeVectors,
            ])

nlp_model_w2v = nlp_pipeline_w2v.fit(spark_df)

processed_w2v = nlp_model_w2v.transform(spark_df)

processed_w2v.count()

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


5000

In [ ]:
processed_w2v.columns

['features',
 'label',
 'tweet',
 'document',
 'token',
 'normalized',
 'cleanTokens',
 'embeddings',
 'sentence_embeddings',
 'finished_sentence_embeddings']

In [ ]:
processed_w2v.show(5)

+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------------------+
|            features|label|               tweet|            document|               token|          normalized|         cleanTokens|          embeddings| sentence_embeddings|finished_sentence_embeddings|
+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------------------+
|[0.02773400582373...|    0|the sellers accep...|[{document, 0, 54...|[{token, 0, 2, th...|[{token, 0, 2, th...|[{token, 4, 10, s...|[{word_embeddings...|[{sentence_embedd...|        [[0.0277340058237...|
|[-0.2108502835035...|    1|the microwave at ...|[{document, 0, 10...|[{token, 0, 2, th...|[{token, 0, 2, th...|[{token, 4, 12, m...|[{word_embeddings...|[{sentence_embedd...|     

In [ ]:
processed_w2v.select('finished_sentence_embeddings').take(1)

[Row(finished_sentence_embeddings=[DenseVector([0.0277, 0.3477, 0.0511, 0.2483, 0.2075, -0.1634, -0.0111, 0.3807, 0.1292, -0.3561, -0.0374, 0.1651, 0.2746, -0.0301, -0.0101, -0.2521, 0.0388, 0.1938, -0.0494, 0.4389, -0.2251, -0.2997, 0.0481, -0.0251, 0.0692, 0.0416, -0.0182, -0.3399, 0.0555, -0.2028, -0.0598, 0.5491, -0.3812, -0.3454, 0.0936, 0.2491, 0.0228, -0.2965, -0.2358, -0.3809, -0.3403, -0.1401, 0.1044, -0.3468, -0.1381, -0.2201, -0.2853, -0.3443, 0.1875, -0.4726, 0.4313, -0.1614, 0.1751, 0.7783, -0.1908, -1.5875, -0.2479, -0.3684, 1.4258, -0.0262, -0.0699, 0.2124, 0.0581, -0.156, 0.5504, -0.156, 0.2305, 0.4646, -0.0853, -0.446, -0.0811, -0.2875, -0.372, -0.195, -0.1316, 0.0803, -0.4341, -0.0463, -0.8069, -0.4025, 0.3565, 0.0571, -0.3571, -0.1755, -1.0731, 0.0975, 0.1237, -0.1974, -0.1266, -0.2944, -0.1871, 0.0597, -0.1935, -0.102, -0.2789, 0.2095, 0.0365, -0.002, 0.5753, 0.6163])])]

In [ ]:
# IF SQLTransformer IS NOT USED INSIDE THE PIPELINE, WE CAN EXPLODE OUTSIDE
from pyspark.sql.functions import explode

# processed_w2v= processed_w2v.withColumn("features", explode(processed_w2v.finished_sentence_embeddings))

In [ ]:
processed_w2v.select("features").take(1)

[Row(features=DenseVector([0.0277, 0.3477, 0.0511, 0.2483, 0.2075, -0.1634, -0.0111, 0.3807, 0.1292, -0.3561, -0.0374, 0.1651, 0.2746, -0.0301, -0.0101, -0.2521, 0.0388, 0.1938, -0.0494, 0.4389, -0.2251, -0.2997, 0.0481, -0.0251, 0.0692, 0.0416, -0.0182, -0.3399, 0.0555, -0.2028, -0.0598, 0.5491, -0.3812, -0.3454, 0.0936, 0.2491, 0.0228, -0.2965, -0.2358, -0.3809, -0.3403, -0.1401, 0.1044, -0.3468, -0.1381, -0.2201, -0.2853, -0.3443, 0.1875, -0.4726, 0.4313, -0.1614, 0.1751, 0.7783, -0.1908, -1.5875, -0.2479, -0.3684, 1.4258, -0.0262, -0.0699, 0.2124, 0.0581, -0.156, 0.5504, -0.156, 0.2305, 0.4646, -0.0853, -0.446, -0.0811, -0.2875, -0.372, -0.195, -0.1316, 0.0803, -0.4341, -0.0463, -0.8069, -0.4025, 0.3565, 0.0571, -0.3571, -0.1755, -1.0731, 0.0975, 0.1237, -0.1974, -0.1266, -0.2944, -0.1871, 0.0597, -0.1935, -0.102, -0.2789, 0.2095, 0.0365, -0.002, 0.5753, 0.6163]))]

In [ ]:
processed_w2v.select('tweet','features','label').show()

+--------------------+--------------------+-----+
|               tweet|            features|label|
+--------------------+--------------------+-----+
|the sellers accep...|[0.02773400582373...|    0|
|the microwave at ...|[-0.2108502835035...|    1|
|RT @Oz_lito: @1st...|[0.01915432699024...|    1|
|it's so amazing h...|[0.18952007591724...|    0|
|  happy day!  #al...|[-0.1074697524309...|    0|
|people who can ge...|[0.02151191607117...|    0|
|@user the library...|[0.04796125739812...|    0|
|somewhere in my t...|[0.09011052548885...|    0|
|back home k.lbvr ...|[0.08463780581951...|    0|
|hating the conser...|[0.01190709043294...|    0|
|Going up to the v...|[0.33191305398941...|    0|
|@user lolol i'm n...|[-0.0720526576042...|    0|
|@CallNeeshCakey p...|[-0.1222900003194...|    0|
|#petal   polar be...|[-0.1342255622148...|    0|
|Love getting nude...|[0.46387749910354...|    1|
|You a hoe if you ...|[-0.0718640014529...|    1|
|best #lawofattrac...|[0.02930671349167...|    0|


In [ ]:
# set seed for reproducibility
(trainingData, testData) = processed_w2v.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 3478
Test Dataset Count: 1522


In [ ]:
## By removing such rows, we can optimize the performance of our machine learning algorithms, as we can skip the computation for those rows where the features vector is all zeros.
from pyspark.sql.functions import udf

@udf("long")
def num_nonzeros(v):
    return v.numNonzeros()

testData = testData.where(num_nonzeros("features") != 0)

In [ ]:
lrModel_w2v = lr.fit(trainingData)

In [ ]:
predictions_w2v = lrModel_w2v.transform(testData)

predictions_w2v.select("tweet","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 60)

+------------------------------------------------------------+-----------------------------------------+-----+----------+
|                                                       tweet|                              probability|label|prediction|
+------------------------------------------------------------+-----------------------------------------+-----+----------+
|                                 morning everyone!   #monday|[0.9700461803788555,0.029953819621144517]|    0|       0.0|
|                 @user @user @user @user   saturday  friends| [0.9688514337163213,0.03114856628367868]|    0|       0.0|
|   i am thankful for vacation days. #thankful #positive     |  [0.967206006437128,0.03279399356287205]|    0|       0.0|
| @user 3 weeks until holiday!!!!!!! #pougal #villa #famil...| [0.9642743078981164,0.03572569210188359]|    0|       0.0|
|          flights booked to see my girl @user this summer!  | [0.9627148800456357,0.03728511995436434]|    0|       0.0|
|  #friday peepsðð

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import pandas as pd

y_true = predictions_w2v.select("label")
y_true = y_true.toPandas()

y_pred = predictions_w2v.select("prediction")
y_pred = y_pred.toPandas()

print(classification_report(y_true.label, y_pred.prediction))
print(accuracy_score(y_true.label, y_pred.prediction))

              precision    recall  f1-score   support

           0       0.86      0.89      0.88       766
           1       0.88      0.85      0.87       753

    accuracy                           0.87      1519
   macro avg       0.87      0.87      0.87      1519
weighted avg       0.87      0.87      0.87      1519

0.8722843976300197


In [ ]:
processed_w2v.select('tweet','cleanTokens.result').show(truncate=50)

+--------------------------------------------------+--------------------------------------------------+
|                                             tweet|                                            result|
+--------------------------------------------------+--------------------------------------------------+
|the sellers accepted my offer. and the process ...|       [sellers, accepted, offer, process, begins]|
|the microwave at work is broken, so my ghetto a...|[microwave, work, broken, ghetto, ass, put, ham...|
|RT @Oz_lito: @1stBlocJeremiah ctfu u bloody but...|[RT, Ozlito, stBlocJeremiah, ctfu, u, bloody, y...|
|it's so amazing how it's hard for disadvantaged...|[amazing, hard, disadvantaged, footballers, set...|
|          happy day!  #altwaystoheal  #healthy    |              [happy, day, altwaystoheal, healthy]|
|people who can get time off work to go to the e...|[people, get, time, work, go, euros, havent, go...|
|@user the library of alexandria wasn't the only...|[user, libra

## Classifier DL + Glove + Basic text processing

In [24]:
spark_df.show()

+-----+--------------------+
|label|               tweet|
+-----+--------------------+
|    0|wow september 1 2...|
|    1|RT @UrbanCrazines...|
|    1|RT @RuNeshaShamia...|
|    0|i am thankful for...|
|    0|justice has been ...|
|    0|#diasoleado ðð...|
|    0|#twinkletwinkle t...|
|    0|5 more days until...|
|    0|@user oh dear, is...|
|    0|#visiting   goril...|
|    0|this makes me #ma...|
|    0|just been to view...|
|    0| @user just read ...|
|    0|#lucky   #dad to ...|
|    1|council staff hav...|
|    1|@SlothNG i can gu...|
|    0|i am fantastic. #...|
|    1|@chandraingram95 ...|
|    0|These peanut butt...|
|    0|@user the dance m...|
+-----+--------------------+
only showing top 20 rows



In [26]:
df=spark_df.toPandas()
df = df.rename({'tweet':'text'}, axis = 1)

In [27]:
# Split the data into train and test sets
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

# Save train and test data as CSV files
train_df.to_csv('train_data.csv', index=False)
test_df.to_csv('test_data.csv', index=False)

In [32]:
document_assembler = DocumentAssembler() \
      .setInputCol("text") \
      .setOutputCol("document")
    
tokenizer = Tokenizer() \
      .setInputCols(["document"]) \
      .setOutputCol("token")
    
normalizer = Normalizer() \
      .setInputCols(["token"]) \
      .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

lemma = LemmatizerModel.pretrained('lemma_antbnc') \
      .setInputCols(["cleanTokens"]) \
      .setOutputCol("lemma")

glove_embeddings = WordEmbeddingsModel().pretrained() \
      .setInputCols(["document",'lemma'])\
      .setOutputCol("embeddings")\
      .setCaseSensitive(False)

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "embeddings"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")

classsifierdl = ClassifierDLApproach()\
      .setInputCols(["sentence_embeddings"])\
      .setOutputCol("class")\
      .setLabelColumn("label")\
      .setMaxEpochs(5)\
      .setEnableOutputLogs(True)

clf_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            lemma, 
            glove_embeddings,
            embeddingsSentence,
            classsifierdl])

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]
glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


In [33]:
trainingData= spark_df = spark.createDataFrame(train_df)

In [34]:
trainingData.show()

+-----+--------------------+
|label|                text|
+-----+--------------------+
|    1|RT @PRIVALEDGE: U...|
|    0|@BasketballPics w...|
|    0|the simple things...|
|    0|#model   i love u...|
|    0|' #urdu poetry #b...|
|    0|  father's day. i...|
|    0|okay i need a sac...|
|    1|Niccas be gassing...|
|    1|RT @gotmollylove:...|
|    1|RT @KMURDER1000: ...|
|    1|@HeelsOverSneaks ...|
|    0|this looks like m...|
|    0|full week now and...|
|    0|disney gator atta...|
|    1|@Groovy_Ear strai...|
|    0|#raw food diet be...|
|    0|Mark Teixeira fin...|
|    0|@user @user @user...|
|    0|okay @user i am l...|
|    0|my beautiful best...|
+-----+--------------------+
only showing top 20 rows



In [35]:
clf_pipelineModel = clf_pipeline.fit(trainingData)

In [36]:
!rm -rf classifier_dl_pipeline_glove

In [37]:
clf_pipeline.save('classifier_dl_pipeline_glove')

In [38]:
clf_pipelineModel = clf_pipeline.fit(trainingData)

In [40]:
testData= spark_df = spark.createDataFrame(test_df)

In [41]:
df = clf_pipelineModel.transform(testData).select('label','text',"class.result").toPandas()

df.head()


,label,text,result
0,0,i hate when people take shit that don't belong...,[1]
1,0,no cock out though.,[1]
2,0,&amp; #grateful that #money comes to me in #...,[0]
3,1,#democrats are the 1st to tell jokes,[1]
4,1,And it's always the black bitches that's const...,[1]


In [42]:
from sklearn.metrics import classification_report, accuracy_score

df['result'] = df['result'].apply(lambda x: x[0])

print(classification_report(df["label"].astype(str), df["result"]))

print(accuracy_score(df.label.astype(str), df.result))

              precision    recall  f1-score   support

           0       0.85      0.90      0.88       730
           1       0.90      0.85      0.88       770

    accuracy                           0.88      1500
   macro avg       0.88      0.88      0.88      1500
weighted avg       0.88      0.88      0.88      1500

0.8753333333333333


## NLP TEST

In [43]:
!pip install nlptest
!pip install johnsnowlabs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.7/486.7 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 639.9/639.9 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.4/95.4 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.6 MB/s eta 0:0

In [44]:
import pandas as pd

In [45]:
trainingData= pd.read_csv("train_data.csv")

In [46]:
trainingData

,label,text
0,1,RT @PRIVALEDGE: Undercover hoes
1,0,@BasketballPics whoever made this twitter does...
2,0,the simple things in life make one happy #smil...
3,0,#model i love u take with u all the time in ...
4,0,' #urdu poetry #beutiful poetry poetry #2 l...
...,...,...
3495,1,Live birds found in Elmo doll at US-Mexico bor...
3496,1,"""but pussy is the root of all drama an attribu..."
3497,1,@tGR0Z you's a nasty lil bitch
3498,0,with le sister! #instatraveling #instamoment #...


In [47]:
traindata = 'train_data.csv'

## Robustness Tests

In [48]:
from nlptest import Harness
harness = Harness(task="text-classification", model=clf_pipelineModel, data=traindata)


In [49]:
harness.configure(
{'defaults': {'min_pass_rate': 0.65},
 'tests': {'robustness': {
                          'uppercase':{'min_pass_rate': 0.60},
                          'add_typo':{'min_pass_rate': 0.90}},

          }
 }
 )

{'defaults': {'min_pass_rate': 0.65},
 'tests': {'robustness': {'uppercase': {'min_pass_rate': 0.6},
   'add_typo': {'min_pass_rate': 0.9}}}}

In [51]:
harness.generate()

Generating testcases...: 100%|██████████| 1/1 [00:00<00:00, 6775.94it/s]


In [52]:
harness.testcases()

,category,test_type,original,test_case,expected_result
0,robustness,uppercase,RT @PRIVALEDGE: Undercover hoes,RT @PRIVALEDGE: UNDERCOVER HOES,0
1,robustness,uppercase,@BasketballPics whoever made this twitter does...,@BASKETBALLPICS WHOEVER MADE THIS TWITTER DOES...,1
2,robustness,uppercase,the simple things in life make one happy #smil...,THE SIMPLE THINGS IN LIFE MAKE ONE HAPPY #SMIL...,1
3,robustness,uppercase,#model i love u take with u all the time in ...,#MODEL I LOVE U TAKE WITH U ALL THE TIME IN ...,1
4,robustness,uppercase,' #urdu poetry #beutiful poetry poetry #2 l...,' #URDU POETRY #BEUTIFUL POETRY POETRY #2 L...,1
...,...,...,...,...,...
6995,robustness,add_typo,Live birds found in Elmo doll at US-Mexico bor...,Live birds found in Elmo doll at US-Mexico bor...,1
6996,robustness,add_typo,"""but pussy is the root of all drama an attribu...","""but pussy is the root of all drama an attribu...",0
6997,robustness,add_typo,@tGR0Z you's a nasty lil bitch,@tGR0Z you's a nasty lil bitch,0
6998,robustness,add_typo,with le sister! #instatraveling #instamoment #...,with le sister! #instatraveling #ipstamoment #...,0


In [53]:
harness.run()

Running testcases... : 100%|██████████| 7000/7000 [49:09<00:00,  2.37it/s]


In [55]:
harness.generated_results()

,category,test_type,original,test_case,expected_result,actual_result,pass
0,robustness,uppercase,RT @PRIVALEDGE: Undercover hoes,RT @PRIVALEDGE: UNDERCOVER HOES,0,0,True
1,robustness,uppercase,@BasketballPics whoever made this twitter does...,@BASKETBALLPICS WHOEVER MADE THIS TWITTER DOES...,1,1,True
2,robustness,uppercase,the simple things in life make one happy #smil...,THE SIMPLE THINGS IN LIFE MAKE ONE HAPPY #SMIL...,1,1,True
3,robustness,uppercase,#model i love u take with u all the time in ...,#MODEL I LOVE U TAKE WITH U ALL THE TIME IN ...,1,1,True
4,robustness,uppercase,' #urdu poetry #beutiful poetry poetry #2 l...,' #URDU POETRY #BEUTIFUL POETRY POETRY #2 L...,1,1,True
...,...,...,...,...,...,...,...
6995,robustness,add_typo,Live birds found in Elmo doll at US-Mexico bor...,Live birds found in Elmo doll at US-Mexico bor...,1,1,True
6996,robustness,add_typo,"""but pussy is the root of all drama an attribu...","""but pussy is the root of all drama an attribu...",0,0,True
6997,robustness,add_typo,@tGR0Z you's a nasty lil bitch,@tGR0Z you's a nasty lil bitch,0,0,True
6998,robustness,add_typo,with le sister! #instatraveling #instamoment #...,with le sister! #instatraveling #ipstamoment #...,0,0,True


## Bias Tests

In [68]:
harness = Harness(task="text-classification", model=clf_pipelineModel, data=traindata)

In [69]:
harness.configure(
{'defaults': {'min_pass_rate': 0.65},
 'tests': {'bias': {'replace_to_black_lastnames': {'min_pass_rate': 0.66}, 
                          'replace_to_white_lastnames':{'min_pass_rate': 0.60}}
          }
 }
 )

{'defaults': {'min_pass_rate': 0.65},
 'tests': {'bias': {'replace_to_black_lastnames': {'min_pass_rate': 0.66},
   'replace_to_white_lastnames': {'min_pass_rate': 0.6}}}}

In [70]:
harness.generate()

Generating testcases...: 100%|██████████| 1/1 [00:00<00:00, 6462.72it/s]


In [71]:
harness.testcases()

,category,test_type,original,test_case,expected_result
0,bias,replace_to_black_lastnames,RT @PRIVALEDGE: Undercover hoes,RT @PRIVALEDGE: Undercover hoes,0
1,bias,replace_to_black_lastnames,@BasketballPics whoever made this twitter does...,@BasketballPics whoever made this twitter does...,1
2,bias,replace_to_black_lastnames,the simple things in life make one happy #smil...,the simple things in life make one happy #smil...,1
3,bias,replace_to_black_lastnames,#model i love u take with u all the time in ...,#model i love u take with u all the time in ...,1
4,bias,replace_to_black_lastnames,' #urdu poetry #beutiful poetry poetry #2 l...,' #urdu poetry #beutiful poetry poetry #2 l...,1
...,...,...,...,...,...
6995,bias,replace_to_white_lastnames,Live birds found in Elmo doll at US-Mexico bor...,Live birds found in Elmo doll at US-Mexico bor...,1
6996,bias,replace_to_white_lastnames,"""but pussy is the root of all drama an attribu...","""but pussy is the root of all drama an attribu...",0
6997,bias,replace_to_white_lastnames,@tGR0Z you's a nasty lil bitch,@tGR0Z you's a nasty lil bitch,0
6998,bias,replace_to_white_lastnames,with le sister! #instatraveling #instamoment #...,with Vandyke sister! #instatraveling #instamom...,0


In [72]:
harness.run()

Running testcases... : 100%|██████████| 7000/7000 [48:27<00:00,  2.41it/s]


In [73]:
harness.generated_results()

,category,test_type,original,test_case,expected_result,actual_result,pass
0,bias,replace_to_black_lastnames,RT @PRIVALEDGE: Undercover hoes,RT @PRIVALEDGE: Undercover hoes,0,0,True
1,bias,replace_to_black_lastnames,@BasketballPics whoever made this twitter does...,@BasketballPics whoever made this twitter does...,1,1,True
2,bias,replace_to_black_lastnames,the simple things in life make one happy #smil...,the simple things in life make one happy #smil...,1,1,True
3,bias,replace_to_black_lastnames,#model i love u take with u all the time in ...,#model i love u take with u all the time in ...,1,1,True
4,bias,replace_to_black_lastnames,' #urdu poetry #beutiful poetry poetry #2 l...,' #urdu poetry #beutiful poetry poetry #2 l...,1,1,True
...,...,...,...,...,...,...,...
6995,bias,replace_to_white_lastnames,Live birds found in Elmo doll at US-Mexico bor...,Live birds found in Elmo doll at US-Mexico bor...,1,1,True
6996,bias,replace_to_white_lastnames,"""but pussy is the root of all drama an attribu...","""but pussy is the root of all drama an attribu...",0,0,True
6997,bias,replace_to_white_lastnames,@tGR0Z you's a nasty lil bitch,@tGR0Z you's a nasty lil bitch,0,0,True
6998,bias,replace_to_white_lastnames,with le sister! #instatraveling #instamoment #...,with Vandyke sister! #instatraveling #instamom...,0,0,True


In [76]:
df=harness.generated_results()

In [78]:
type(df)

pandas.core.frame.DataFrame

In [86]:
grouped_data = df.groupby(['test_type', 'pass'])

In [82]:
grouped_counts = grouped_data.size()

In [83]:
grouped_counts 

test_type                   pass 
replace_to_black_lastnames  False     244
                            True     3256
replace_to_white_lastnames  False     163
                            True     3337
dtype: int64